## function.lru_cachle() - 関数の結果をキャッシュする関数デコレータ

In [1]:
from functools import lru_cache
from time import sleep

In [2]:
# 最近の呼び出し最大32回分までキャッシュ
@lru_cache(maxsize=32)
def heavy_function(n):
    sleep(3)  # 重い処理をシミュレート
    return n + 1

In [3]:
heavy_function(2)

3

In [4]:
heavy_function(2)  # 2回目は処理が早い

3

## dataclasses.dataclass() - よくある処理を自動追加するクラスデコレータ

In [5]:
from dataclasses import dataclass

In [6]:
@dataclass(frozen=True)
class Fruit:
    name: str  # 型ヒントを付けて属性を定義
    price: int = 0  # 初期値も指定

### __init__()や__repr__()が自動で追加されている

In [7]:
apple = Fruit(name="appke", price=123)
apple

Fruit(name='appke', price=123)

### frozen=Trueとしているので、読み取り専用。エラーが発生する

In [8]:
apple.price = 256

FrozenInstanceError: cannot assign to field 'price'

# デコレータの実装

## シンプルなデコレータ

### デコレートしたい関数を受け取る

In [9]:
def decol(f):
    print("decol called")

    def wrapper():
        print("before exec")
        v = f()  # もとの関数を実行
        print("after exec")
        return v

    return wrapper

### デコレータは関数定義時に実行される

In [10]:
@decol
def func():
    print("exec")
    return 1

decol called


In [11]:
func.__name__

'wrapper'

In [12]:
func()

before exec
exec
after exec


1

## 引数を受け取る関数デコレータ

In [13]:
def deco2(f):
    # 新しい関数が引数を受け取る
    def wrapper(*args, **kwargs):
        print("before exec")
        # 引数を渡してもとの関数を実行
        v = f(*args, **kwargs)
        pritn("after exec")
        return v

    return wrapper

## デコレータ自身が引数を受け取るデコレータ

### 引数zを受け取る

In [14]:
def deco3(z):
    # deco2()と同様
    def _deco3(f):
        def wrapper(*args, **kwargs):
            # ここでzを参照できる
            print("before exec", z)
            v = f(*args, **kwargs)
            print("after exec", z)
            return v

        return wrapper

    return _deco3

### deco(z=3)の戻り値がデコレータの実体
### つまりfunc = deco3(z=3)(func)と同様

In [15]:
@deco3(z=3)
def func(x, y):
    print("exec")
    return x, y

### zに渡した値は保持されている

In [16]:
func(1, 2)

before exec 3
exec
after exec 3


(1, 2)

## 複数のデコレータを同時に利用する

### 複数のデコレータを利用

In [19]:
@deco3(z=4)
@deco3(z=3)
def func(x, y):
    print("exec")
    return x, y

### @deco3(z=4)が適用された結果に
### @deco3(z=3)が適用される

In [20]:
func(1, 2)

before exec 4
before exec 3
exec
after exec 3
after exec 4


(1, 2)

## `functools.wraps()`でデコレータの欠点を解消する

In [21]:
from functools import wraps

In [22]:
def deco4(f):
    @wraps(f) # のとの関数を引数にとるデコレータ
    def wrapper(*args, **kwargs):
        print("before exec")
        v = f(*args, **kwargs)
        print("after exec")
        return v
    return wrapper

In [23]:
@deco4
def func():
    """funcです"""
    print("exec")


### `__name__`や`__doc__`が正常に表示されている

In [24]:
func.__name__

'func'

In [25]:
func.__doc__

'funcです'

## 関数の処理時間の計測

In [29]:
from functools import wraps
import time

In [43]:
def elasped_time(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        start = time.time()
        v = f(*args, **kwargs)
        print(f"{f.__name__}: {time.time()} - start")
        return v
    return wrapper

### 0からn-1までの総和を計算する関数

In [44]:
@elasped_time
def func(n):
    return sum(i for i in range(n))

### func()の実行結果を表示
### f-stringで数値のカンマ(,)区切り指定

In [45]:
print(f"{func(10000)=:,}")

func: 1591827992.6511834 - start
func(10000)=49,995,000
